In [1]:
import warnings

import numpy as np
import pandas as pd
import useful_rid_code as rid
from openpyxl import load_workbook

warnings.simplefilter("ignore")

# RID 

## Fill-in OSPAR reporting template (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 2001


## 1. Get summary data

The notebooks `summary_table_{year}.ipynb` calculate summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [4]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)
del mon_df["new_rid_group"]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "old_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("old_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "old_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"], mon_df["mean_q_1000m3/day"]

# Convert units
mon_df["Hg_kg"] = mon_df["Hg_kg"] / 1000.0  # kg to tonnes
mon_df["NH4-N_tonnes"] = mon_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["NO3-N_tonnes"] = mon_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTN_tonnes"] = mon_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTP_tonnes"] = mon_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["PO4-P_tonnes"] = mon_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["SPM_tonnes"] = mon_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

As   Cd    Cr    Cu  NH4-N  NO3-N    Ni  \
ospar_region        old_rid_group                                              
LOFOTEN-BARENTS SEA rid_108         0.0  0.0   1.0   5.0    0.0    0.0   4.0   
                    rid_11          1.0  0.0   1.0   1.0    0.0    0.0   1.0   
                    rid_36          1.0  0.0   4.0  12.0    0.0    0.0  35.0   
NORTH SEA           rid_108         1.0  0.0   1.0   5.0    0.0    2.0   3.0   
                    rid_11          0.0  0.0   0.0   1.0    0.0    0.0   0.0   
                    rid_36          1.0  0.0   0.0   4.0    0.0    3.0   1.0   
NORWEGIAN SEA2      rid_108         1.0  0.0   1.0   4.0    0.0    1.0   2.0   
                    rid_11          1.0  0.0   2.0  12.0    0.0    1.0   3.0   
                    rid_36          2.0  0.0   4.0  22.0    0.0    2.0  12.0   
SKAGERAK            rid_108         3.0  0.0   4.0   7.0    0.0    1.0   2.0   
                    rid_11          8.0  1.0  11.0  62.0    1.0   14.0  29.0   
                    rid_36          1.0  0.0   1.0   5.0    0.0    2.0   2.0   
NORWAY              rid_108         6.0  0.0   7.0  22.0    1.0    4.0  11.0   
                    rid_11         10.0  1.0  14.0  76.0    2.0   15.0  33.0   
                    rid_36          5.0  1.0   9.0  44.0    1.0    6.0  50.0   

                                   PO4-P    Pb    SPM      SiO2       TOC  \
ospar_region        old_rid_group                                           
LOFOTEN-BARENTS SEA rid_108          0.0   0.0    6.0   15826.0   17461.0   
                    rid_11           0.0   0.0    3.0   13813.0   10548.0   
                    rid_36           0.0   0.0   10.0   42416.0   52922.0   
NORTH SEA           rid_108          0.0   1.0   13.0   13248.0   17134.0   
                    rid_11           0.0   0.0    3.0    2031.0    2521.0   
                    rid_36           0.0   3.0   30.0   13557.0   33589.0   
NORWEGIAN SEA2      rid_108          0.0   0.0   18.0   10287.0   26748.0   
                    rid_11           0.0   0.0   13.0   11787.0   16965.0   
                    rid_36           0.0   1.0   97.0   26563.0   99374.0   
SKAGERAK            rid_108          0.0   1.0   12.0    7306.0   18964.0   
                    rid_11           0.0  12.0  315.0  180907.0  217123.0   
                    rid_36           0.0   1.0   10.0   14160.0   35491.0   
NORWAY              rid_108          0.0   3.0   49.0   46668.0   80306.0   
                    rid_11           0.0  12.0  334.0  208537.0  247157.0   
                    rid_36           0.0   6.0  148.0   96697.0  221376.0   

                                   TOTN     Zn   Ag   Hg  TOTP  
ospar_region        old_rid_group                               
LOFOTEN-BARENTS SEA rid_108         1.0    6.0  0.0  0.0   0.0  
                    rid_11          1.0    2.0  0.0  0.0   0.0  
                    rid_36          3.0   48.0  0.0  0.0   0.0  
NORTH SEA           rid_108         3.0   24.0  0.0  0.0   0.0  
                    rid_11          1.0    1.0  0.0  0.0   0.0  
                    rid_36          4.0   42.0  0.0  0.0   0.0  
NORWEGIAN SEA2      rid_108         2.0    7.0  0.0  0.0   0.0  
                    rid_11          1.0   64.0  0.0  0.0   0.0  
                    rid_36          5.0   34.0  0.0  0.0   0.0  
SKAGERAK            rid_108         2.0    8.0  0.0  0.0   0.0  
                    rid_11         24.0  187.0  0.0  0.0   1.0  
                    rid_36          3.0   41.0  0.0  0.0   0.0  
NORWAY              rid_108         8.0   45.0  0.0  0.0   0.0  
                    rid_11         26.0  253.0  0.0  0.0   1.0  
                    rid_36         15.0  164.0  0.0  0.0   0.0

### 1.2. Unmonitored areas

In [5]:
# Read data
in_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Add missing columns if necessary
cols = ["sew_Hg", "sew_S.P.M.", "ind_Hg"]
for col in cols:
    if col not in umon_df.columns:
        umon_df[col] = 0

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg,fish_Cu,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,296346.0,10906.0,903.0,2437.0,179.0,19447.0,3825.0,29721.0,642.0,542.0,...,8.0,0.0,7.0,23.0,15.0,1.0,9.0,361.0,0,0
LOFOTEN-BARENTS SEA,95366.0,1058.0,140.0,0.0,0.0,4888.0,969.0,4545.0,94.0,84.0,...,3.0,0.0,0.0,1.0,1.0,0.0,0.0,89.0,0,0
NORTH SEA,85866.0,3060.0,286.0,526.0,47.0,8007.0,1575.0,11226.0,189.0,172.0,...,5.0,0.0,1.0,21.0,12.0,0.0,4.0,149.0,0,0
NORWEGIAN SEA2,103964.0,2530.0,340.0,676.0,65.0,6551.0,1281.0,10914.0,271.0,204.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0,0,0
SKAGERAK,11150.0,4258.0,137.0,1234.0,67.0,2.0,0.0,3036.0,88.0,82.0,...,0.0,0.0,6.0,1.0,2.0,1.0,4.0,0.0,0,0


### 1.3. Loads for 11 main rivers

In [6]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df["new_rid_group"]

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df["station_code"], rid11_df["station_name"]
del rid11_df["old_rid_group"], rid11_df["ospar_region"]
del rid11_df["mean_q_1000m3/day"]

cols = [i for i in rid11_df.columns if i.split("_")[1] != "Est"]
rid11_df = rid11_df[cols]

# Convert units
rid11_df["Hg_kg"] = rid11_df["Hg_kg"] / 1000.0  # kg to tonnes
rid11_df["NH4-N_tonnes"] = rid11_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["NO3-N_tonnes"] = rid11_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTN_tonnes"] = rid11_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTP_tonnes"] = rid11_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["PO4-P_tonnes"] = rid11_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["SPM_tonnes"] = rid11_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split("_")[0] for i in rid11_df.columns]

rid11_df.head()

,As,Cd,Cr,Cu,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,Zn,Ag,Hg,TOTP
station_id,,,,,,,,,,,,,,,,,
29612,1.753694,0.164140,2.576537,11.112039,0.309779,2.597325,5.423230,0.033214,0.784610,27.403467,35879.345494,42075.333058,4.550678,37.280884,0.012751,0.009037,0.071204
29779,0.713106,0.037622,1.362461,1.481883,0.024575,0.206778,1.028363,0.031023,0.061184,3.286396,13812.900836,10548.035666,0.616057,1.773164,0.004275,0.002088,0.022353
29821,0.138545,0.004618,0.115454,0.346363,0.025400,0.177800,0.000000,0.000000,0.000000,1.616362,1781.296392,1616.361541,0.376381,1.062180,0.006390,0.002042,0.008269
29782,0.613664,0.030507,1.208569,1.925156,0.018921,0.280051,1.254453,0.012622,0.183242,10.513567,6535.120245,10265.167270,0.883971,35.675359,0.006400,0.004352,0.017433
36225,0.098388,0.007826,0.288455,1.071084,0.056349,0.213099,0.225844,0.014087,0.084971,1.453454,1724.978674,1408.732584,0.390420,1.632341,0.002253,0.000271,0.024110


## 2. Fill-in template

The template is usually sent each year by Csilla at NIBIO. However, it doesn't seem to change, so old versions can also be used e.g. here:

    ../../../Results/OSPAR/Blank_Template

In [7]:
# Copy of template to update
temp_path = rid.copy_ospar_template(year)

In [8]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """Update the OSPAR template for point source data.

    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "S.P.M.",
        "TOC": "TOC",
        "PO4-P": "po4",
        "P-Total": "p",
        "NO3-N": "no3",
        "NH4-N": "nh4",
        "N-Total": "n",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names
    names_dict = {
        "Norwegian Sea (NO)": "NORWEGIAN SEA2",
        "Barents Sea (NO)": "LOFOTEN-BARENTS SEA",
        "Skagerrak (NO)": "SKAGERAK",
        "North Sea (NO)": "NORTH SEA",
        "Norway Total": "NORWAY",
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            try:
                val = df.loc[names_dict[reg], "%s_%s" % (src, par_dict[par])]
            except KeyError:
                val = 0

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [9]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split("_")[0] == "sew"]
sew_df = umon_df[cols].copy()

# Convert units
sew_df["sew_Hg"] = sew_df["sew_Hg"] / 1000.0  # kg to tonnes
sew_df["sew_nh4"] = sew_df["sew_nh4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_no3"] = sew_df["sew_no3"] / 1000.0  # tonnes to ktonnes
sew_df["sew_n"] = sew_df["sew_n"] / 1000.0  # tonnes to ktonnes
sew_df["sew_po4"] = sew_df["sew_po4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_p"] = sew_df["sew_p"] / 1000.0  # tonnes to ktonnes
sew_df["sew_S.P.M."] = sew_df["sew_S.P.M."] / 1000.0  # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,
NORWAY,10.906221,0.903499,0.542099,0.545311,8.179665,0.0,0.0
LOFOTEN-BARENTS SEA,1.058254,0.140468,0.084281,0.052913,0.793691,0.0,0.0
NORTH SEA,3.060404,0.285956,0.171574,0.153020,2.295303,0.0,0.0
NORWEGIAN SEA2,2.529612,0.339911,0.203946,0.126481,1.897209,0.0,0.0
SKAGERAK,4.257951,0.137164,0.082298,0.212898,3.193463,0.0,0.0


In [10]:
# Update sheet 5a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
]

update_spreadsheet_point_sources(temp_path, "5a", pars, "sew", sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [11]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split("_")[0] == "ind"]
ind_df = umon_df[cols].copy()

# Convert units
ind_df["ind_Hg"] = ind_df["ind_Hg"] / 1000.0  # kg to tonnes
ind_df["ind_nh4"] = ind_df["ind_nh4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_no3"] = ind_df["ind_no3"] / 1000.0  # tonnes to ktonnes
ind_df["ind_n"] = ind_df["ind_n"] / 1000.0  # tonnes to ktonnes
ind_df["ind_po4"] = ind_df["ind_po4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_p"] = ind_df["ind_p"] / 1000.0  # tonnes to ktonnes
ind_df["ind_S.P.M."] = ind_df["ind_S.P.M."] / 1000.0  # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.436517,0.178687,0.107212,0.121826,1.827388,647.166752,273.826,0.906865,8.276829,0.210815,7.129067,22.724625,15.362217,0.697193,0.008762
LOFOTEN-BARENTS SEA,0.000000,0.000224,0.000135,0.000000,0.000000,240.352402,0.450,0.553000,3.201008,0.042002,0.102500,0.716070,1.332000,0.000000,0.000206
NORTH SEA,0.525918,0.046602,0.027961,0.026296,0.394439,10.282944,172.260,0.111027,4.966713,0.153891,0.635716,20.812786,11.932115,0.014321,0.004406
NORWEGIAN SEA2,0.676109,0.064695,0.038817,0.033805,0.507082,394.334984,83.214,0.000290,0.006181,0.000817,0.055088,0.099180,0.013699,0.000000,0.000114
SKAGERAK,1.234490,0.067165,0.040299,0.061725,0.925867,2.196422,17.902,0.242548,0.102926,0.014105,6.335762,1.096589,2.084402,0.682872,0.004037


In [12]:
# Update sheet 5b
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5b", pars, "ind", ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [13]:
# Get fish data
cols = [i for i in umon_df.columns if i.split("_")[0] == "fish"]
fish_df = umon_df[cols].copy()

# Convert units
fish_df["fish_nh4"] = fish_df["fish_nh4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_no3"] = fish_df["fish_no3"] / 1000.0  # tonnes to ktonnes
fish_df["fish_n"] = fish_df["fish_n"] / 1000.0  # tonnes to ktonnes
fish_df["fish_po4"] = fish_df["fish_po4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_p"] = fish_df["fish_p"] / 1000.0  # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,19.446807,3.825264,2.639432,2.139149,15.557446,360.981679
LOFOTEN-BARENTS SEA,4.887601,0.968864,0.668516,0.537636,3.910081,89.251489
NORTH SEA,8.006700,1.574646,1.086505,0.880737,6.405360,149.413728
NORWEGIAN SEA2,6.550954,1.281470,0.884214,0.720605,5.240763,122.287532
SKAGERAK,0.001551,0.000284,0.000196,0.000171,0.001241,0.028930


In [14]:
# Update sheet 5c
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total", "Cu"]

update_spreadsheet_point_sources(temp_path, "5c", pars, "fish", fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [15]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split("_")[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby("region").sum()

td_df.columns = ["td_" + i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.553000,0.042002,0.000000,89.353989,0.000206,1.332000,3.201008,240.352402,0.450,0.716070,5.945856,4.703772,0.590549,1.109556,0.752932
NORTH SEA,0.111027,0.153891,0.014321,150.049445,0.004406,11.932115,4.966713,10.282944,172.260,20.812786,11.593023,9.095102,1.060053,1.907204,1.286041
NORWAY,0.906865,0.210815,0.697193,368.110746,0.008762,15.362217,8.276829,647.166752,273.826,22.724625,32.789545,25.564499,2.806286,4.907450,3.288744
NORWEGIAN SEA2,0.000290,0.000817,0.000000,122.342620,0.000114,0.013699,0.006181,394.334984,83.214,0.099180,9.756675,7.645054,0.880891,1.686076,1.126978
SKAGERAK,0.242548,0.014105,0.682872,6.364692,0.004037,2.084402,0.102926,2.196422,17.902,1.096589,5.493992,4.120571,0.274793,0.204614,0.122794


In [16]:
# Update sheet 5e
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5e", pars, "td", td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [17]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby("ospar_region").sum()
trib_df = mon_df[mon_df["old_rid_group"] != "rid_11"].groupby("ospar_region").sum()

In [18]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """Update the OSPAR template for monitored rivers.

    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "SPM",
        "TOC": "TOC",
        "PO4-P": "PO4-P",
        "P-Total": "TOTP",
        "NO3-N": "NO3-N",
        "NH4-N": "NH4-N",
        "N-Total": "TOTN",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names and rows
    names_dict = {
        "Norwegian Sea (NO)": ("NORWEGIAN SEA2", "tot"),
        "Barents Sea (NO)": ("LOFOTEN-BARENTS SEA", "tot"),
        "Skagerrak (NO)": ("SKAGERAK", "tot"),
        "North Sea (NO)": ("NORTH SEA", "tot"),
        "Norway Total": ("NORWAY", "tot"),
        "Tributary Rivers - Norwegian Sea": ("NORWEGIAN SEA2", "trib"),
        "Tributary Rivers - Barents Sea": ("LOFOTEN-BARENTS SEA", "trib"),
        "Tributary Rivers - Skagerak": ("SKAGERAK", "trib"),
        "Tributary Rivers - North Sea": ("NORTH SEA", "trib"),
        "Orkla": (29778, "main"),
        "Vefsna": (29782, "main"),
        "Alta": (29779, "main"),
        "Glomma": (29617, "main"),
        "Drammenselva": (29612, "main"),
        u"Numedalslågen": (29615, "main"),
        "Skienselva": (29613, "main"),
        "Otra": (29614, "main"),
        "Orreelva": (29783, "main"),
        "Vosso": (29821, "main"),
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]

        # Get df
        df = df_dict[df_name]

        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [19]:
# Update sheet 6a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]
df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6a", pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [20]:
# Get diff data
cols = [i for i in umon_df.columns if i.split("_")[0] == "diff"]
diff_df = umon_df[cols].copy()

# Convert units
diff_df["diff_nh4"] = diff_df["diff_nh4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_no3"] = diff_df["diff_no3"] / 1000.0  # tonnes to ktonnes
diff_df["diff_n"] = diff_df["diff_n"] / 1000.0  # tonnes to ktonnes
diff_df["diff_po4"] = diff_df["diff_po4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_p"] = diff_df["diff_p"] / 1000.0  # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,29.720599,0.641955,0.157921,18.575375,1.634633
LOFOTEN-BARENTS SEA,4.544765,0.093727,0.023057,2.840478,0.249962
NORTH SEA,11.226111,0.189090,0.046516,7.016320,0.617436
NORWEGIAN SEA2,10.913908,0.270690,0.066590,6.821192,0.600265
SKAGERAK,3.035816,0.088449,0.021758,1.897385,0.166970


In [21]:
# Update sheet 6b
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total"]

update_spreadsheet_point_sources(temp_path, "6b", pars, "diff", diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [22]:
# Rename cols in diff_df
col_map = {
    "diff_n": "TOTN",
    "diff_p": "TOTP",
    "diff_po4": "PO4-P",
    "diff_no3": "NO3-N",
    "diff_nh4": "NH4-N",
}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6c", pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [23]:
## Standardise col names
# diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
# td_df.columns = [i.split('_')[1] for i in td_df.columns]
# td_df.index.name = 'ospar_region'
#
# col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
# for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#
## Reset index
# diff_df.reset_index(inplace=True)
# tot_df.reset_index(inplace=True)
# td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
# tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
# tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
# tot_df

In [24]:
## Update sheet 6c
# pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N',
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P',
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
# update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [25]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/concs_and_flows_rid_11_{year}.csv"
conc_df = pd.read_csv(in_csv, index_col=0, encoding="utf-8")

cols = [
    "Hg_ng/l",
    "NH4-N_µg/l N",
    "NO3-N_µg/l N",
    "TOTN_µg/l N",
    "TOTP_µg/l P",
    "PO4-P_µg/l P",
    "TOC_mg C/l",
]
for col in cols:
    if col not in conc_df.columns:
        par = col.split("_")[0]
        conc_df[col] = np.nan
        conc_df[f"{par}_flag"] = np.nan

# Convert units
conc_df["Hg_ng/l"] = conc_df["Hg_ng/l"] / 1000  # ng to ug
conc_df["NH4-N_µg/l N"] = conc_df["NH4-N_µg/l N"] / 1000  # ug to mg
conc_df["NO3-N_µg/l N"] = conc_df["NO3-N_µg/l N"] / 1000  # ug to mg
conc_df["TOTN_µg/l N"] = conc_df["TOTN_µg/l N"] / 1000  # ug to mg
conc_df["TOTP_µg/l P"] = conc_df["TOTP_µg/l P"] / 1000  # ug to mg
conc_df["PO4-P_µg/l P"] = conc_df["PO4-P_µg/l P"] / 1000  # ug to mg
conc_df["TOC_mg C/l"] = conc_df["TOC_mg C/l"] * 1000  # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split("_")[1] == "flag"]
lod_df = conc_df[cols]
lod_df.columns = [i.split("_")[0] for i in lod_df.columns]

# Get vals
cols = [
    i
    for i in conc_df.columns
    if ((i.split("_")[0] in lod_df.columns) and (i.split("_")[1] != "flag"))
]
conc_df = conc_df[cols]
conc_df.columns = [i.split("_")[0] for i in conc_df.columns]

# Rename
col_dict = {
    "SPM": "S.P.M.",
    "TOTN": "n",
    "NH4-N": "nh4",
    "NO3-N": "no3",
    "TOTP": "p",
    "PO4-P": "po4",
}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {
    "SPM": "S.P.M.",
    "TOC": "TOC",
    "PO4-P": "po4",
    "P-Total": "p",
    "NO3-N": "no3",
    "NH4-N": "nh4",
    "N-Total": "n",
    "As": "As",
    "Pb": "Pb",
    "Cd": "Cd",
    "Cu": "Cu",
    "Zn": "Zn",
    "Ni": "Ni",
    "Total Cr": "Cr",
    "Hg": "Hg",
}

# Map names to stns
names_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Alta": 29779,
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Orreelva": 29783,
    "Vosso": 29821,
}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb["7"]

# Get row numbers
row_dict = {}
for item in ws["B12" : "B%s" % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws["E9":"AK9"][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        if par_dict[par] not in conc_df.columns:
            conc_df[par_dict[par]] = np.nan
            lod_df[par_dict[par]] = np.nan

        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].values
        lods = lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]].fillna("0").values
        vals[(lods == "<")] = 0
        val = np.nanmean(vals)

        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].dropna())
        if n_samp == 0:
            n_samp = np.nan
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]])).sum()
        try:
            pct_lod = 100 * float(n_lod) / float(n_samp)
            if pct_lod < 30:
                val = "Yes"
            else:
                val = "No"
        except ZeroDivisionError:
            val = np.nan

        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 